In [2]:
class Bbox(object):
	
	_var_names = None
	
	def __init__(self, items, attr = None):
		if not Bbox._var_names:
			self.image_id = items[0]
			self.x_1 = int(items[1])
			self.y_1 = int(items[2])
			self.width = int(items[3])
			self.height = int(items[4])
			self._columns = None
			self.attr = attr if attr.image_id == items[0] else None
			
		elif Bbox._var_names:
			# setattr(self, "_vars", Bbox._var_names)
			setattr(self, Bbox._var_names[0], items[0])
			setattr(self, Bbox._var_names[1], int(items[1]))
			setattr(self, Bbox._var_names[2], int(items[2]))
			setattr(self, Bbox._var_names[3], int(items[3]))
			setattr(self, Bbox._var_names[4].replace('\n', ''), int(items[4]))
			self.attr = attr if attr.image_id == items[0] else None
	
	def set_vars(name_list):
		name_list[-1] = name_list[-1].replace('\n', '')
		Bbox._var_names = name_list
	
	@property
	def vars(self):
		return self._vars

	@property
	def bbox_pt1(self):
		return (self.x_1, self.y_1)

	@property
	def bbox_pt2(self):
		return (self.width, self.height)
	
	def del_none(a_list):
		a_list = [i for i in a_list if len(i) > 0]
		return a_list
	
	def read_text(txt_file):
		_txt = open(txt_file, 'r')
		content = _txt.readlines()
		_txt.close()
		del content[0] # always is a number
		content = list(map(lambda x: x.split(' '), content))
		content = [list(map(lambda y: y.replace('\n', ''), x)) for x in content]
		content = list(map(Bbox.del_none, content))
		return content

class Attributes(object):
	_var_names = None
	
	def __init__(self, items):
		assert Attributes._var_names, "Var names must be given"
		
		# setattr(self, "_vars", Attributes._var_names)
		setattr(self, "image_id", items[0])
		
		for i in range(len(Attributes._var_names)):
			setattr(self, Attributes._var_names[i], int(items[i+1]))
	
	def set_vars(_vars):
		_vars[-1] = _vars[-1].replace('\n', '')
		Attributes._var_names = _vars
	
	@property
	def vars(self):
		return self._var_names
	
	def del_none(a_list):
		a_list = [i for i in a_list if len(i) > 0]
		return a_list
	
	def read_text(txt_file):
		_txt = open(txt_file, 'r')
		content = _txt.readlines()
		_txt.close()
		del content[0] # always is a number
		content = list(map(lambda x: x.split(' '), content))
		content = [list(map(lambda y: y.replace('\n', ''), x)) for x in content]
		content = list(map(Attributes.del_none, content))
		
		return content
		


In [3]:
import os

__MAXVALUE__ = 202598
_dir_celebA = './celebA'

print("Loading data...")
bboxes_data = Bbox.read_text('./celebA/Anno/list_bbox_celeba.txt')
attr_data = Attributes.read_text('./celebA/Anno/list_attr_celeba.txt')
print("Done with data!")

print("Configuring data headers...")
Bbox.set_vars(bboxes_data[0])
Attributes.set_vars(attr_data[0])
print("Done with headers!")

del attr_data[0], bboxes_data[0]

bboxes = []

print("Baking data...")
for i in range(len(bboxes_data)):
	bboxes.append(Bbox(bboxes_data[i], Attributes(attr_data[i])))
bboxes = tuple(bboxes)
print("The cake is done!")

Loading data...
Done with data!
Configuring data headers...
Done with headers!
Baking data...
The cake is done!


In [4]:
# NOTAS:
# Las imagenes cargadas desde img_align_celeba estan recortadas con los bboxes y ajustadas a 218x178
#
# TODO:
# rescatar 1000 imagenes masculinas
# rescatar 1000 imagenes femeninas

In [5]:
im_male = [obj for obj in bboxes if obj.attr.Male == 1]
im_female = [obj for obj in bboxes if obj.attr.Male == -1]

name_male = [obj.image_id for obj in im_male]
name_female = [obj.image_id for obj in im_female]

true_male, true_female = name_male[:10000], name_female[:10000]

print("all images length:", len(im_male)+len(im_female))
print("name_male:", len(name_male), "name_female:", len(name_female))
print("first url male:", true_male[0])

all images length: 202599
name_male: 84434 name_female: 118165
first url male: 000003.jpg


In [4]:
# ubuntu dirs
male_dir = '/home/choppy/Desktop/CNN/male_female_cnn/images/male/'
female_dir = '/home/choppy/Desktop/CNN/male_female_cnn/images/female/'


if not os.path.isdir(male_dir):
    os.makedirs(male_dir)
if not os.path.isdir(female_dir):
    os.makedirs(female_dir)

In [17]:
from zipfile import ZipFile
import shutil

zip_dir = os.path.join(_dir_celebA, 'img_align_celeba.zip')

with ZipFile(zip_dir, 'r') as zzip:
    print("uncompressing...")
    for member in zzip.namelist():
        filename = os.path.basename(member)
        # skip directories
        if not filename:
            continue
        # copy files
        source = zzip.open(member)
        if filename in name_male:
            target = open(os.path.join(male_dir, filename), "wb")
        else:
            target = open(os.path.join(female_dir, filename), "wb")
        with source, target:
            shutil.copyfileobj(source, target)
    print("Done!")
    zzip.close()
    

uncompressing...
Done!
